In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_excel("synthesied_v6_5000_trajectories.xlsx")
df.head()

,Unnamed: 0,dropout,marks_mean,marks_std,marks_min,marks_max,attendance_array,attempts_array,fee_late_due_date_array,slope_marks,pct_change_marks,momentum_marks,deviation_marks,slope_attendance,deviation_attendance
0,0,0,"[11.48377342044787, 14.22873691806728, 21.5875...","[5.0973751350538565, 1.2007581106634673, 2.211...","[8.997065985071377, 21.058817120487312, 11.400...","[21.31255440855106, 20.716357874525812, 28.824...","[64.58044402810233, 53.43046544444715, 44.9280...","[6, 9, 6, 4, 12, 6, 8, 10]","[7.973242413429928, 3.5361369460137624, -0.436...",0.664059,135.072845,15.975006,5.834347,-1.728972,18.687905
1,1,1,"[13.54276620147063, 18.057259086376117, 13.981...","[8.31530651344457, 0.08060355815685138, 3.5399...","[13.927314044696685, 7.589154077474826, 8.8385...","[23.4354844225621, 8.507113716809839, 32.08521...","[48.340487552573805, 61.336538221298845, 44.94...","[6, 9, 16, 16, 12, 12, 8, 8]","[4.294243744628428, 9.776566720417714, 2.00592...",0.073812,224.522953,7.378381,4.433661,0.021574,14.635799
2,2,0,"[15.18615367227953, 6.61167063764647, 9.203825...","[4.319462241803564, 4.215309042229679, 3.03057...","[11.670362960048848, 9.473938897143714, -4.413...","[21.526081980714107, 27.118354201948087, 26.02...","[59.58874364439332, 40.917537938804756, 60.660...","[6, -4, 18, 12, 5, 13, 13, 7]","[9.006599067536754, -0.16648976946970556, 0.25...",1.133632,26.210865,3.740640,6.623146,-2.032535,18.749442
3,3,1,"[19.496558048260574, 26.886545240864937, 13.11...","[6.507582102027443, 4.038279672401515, 3.65316...","[2.434331076087717, -2.70087228645005, 17.4783...","[21.722265585127353, 26.726180422087033, 26.92...","[71.51309159112597, 46.10682856287723, 78.7662...","[6, 10, 15, 14, 8, 6, 0, 1]","[2.055455009469288, 1.820822606115639, 8.19228...",0.033953,87.296550,6.541442,4.775767,-11.407597,46.381068
4,4,0,"[7.039197304879732, 28.583196430282968, 19.324...","[4.472284884487855, 0.8473459600460194, 1.1320...","[14.170989301045996, 16.42057588856665, 16.288...","[20.777530423732387, 17.36261314235881, 15.746...","[72.73482963569514, 47.51471856962135, 37.2972...","[6, 11, 5, 8, 7, 6, -2, 1]","[-1.6407646429456868, 1.9028833127139204, -0.6...",-1.159610,-242.814684,17.558274,13.623280,0.523698,10.163898


In [ ]:
X = df[["marks_mean", "marks_std", "marks_min", "marks_max", "attendance_array", "attempts_array" ,"fee_late_due_date_array" , "slope_marks", "pct_change_marks", "momentum_marks", "deviation_marks", "slope_attendance", "deviation_attendance"]]

y = df["dropout"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
INPUT_DF = df
SEM = 8
array_columns = [
    "marks_mean",
    "marks_std",
    "marks_min",
    "marks_max",
    "attendance_array",
    "attempts_array",
    "fee_late_due_date_array"
]
TARGET = "dropout"

In [ ]:
df = INPUT_DF.copy()
print("Initial shape:", df.shape)
print("Initial dtypes:\n", df.dtypes)

def is_list_like_cell(x):
    return isinstance(x, (list, tuple)) or (isinstance(x, str) and x.strip().startswith("[") and x.strip().endswith("]"))

import ast
def parse_if_list(x):
    if isinstance(x, (list, tuple)):
        return list(x)
    if isinstance(x, str) and x.strip().startswith("[") and x.strip().endswith("]"):
        try:
            return list(ast.literal_eval(x))
        except:
            return []
    return [x] if not pd.isna(x) else []

for col in array_columns:
    if col not in df.columns:
        print(f"skip missing: {col}")
        continue

    lists = df[col].apply(parse_if_list).apply(lambda L: [float(x) if (x is not None and x != '') else 0.0 for x in L[:SEM]] + [0.0] * max(0, SEM - len(L)))

    for i in range(SEM):
        new_col = f"{col}_{i+1}"
        df[new_col] = lists.apply(lambda L: L[i])

    df.drop(columns=[col], inplace=True)

Initial shape: (5000, 15)
Initial dtypes:
 Unnamed: 0                   int64
dropout                      int64
marks_mean                  object
marks_std                   object
marks_min                   object
marks_max                   object
attendance_array            object
attempts_array              object
fee_late_due_date_array     object
slope_marks                float64
pct_change_marks           float64
momentum_marks             float64
deviation_marks            float64
slope_attendance           float64
deviation_attendance       float64
dtype: object


In [ ]:
# ---- Ensure numeric types for other relevant columns ----
# convert attendance-like columns if present (example: sometimes you used 'attendance_array' now expanded)
for c in list(df.columns):
    if df[c].dtype == object and df[c].apply(lambda v: isinstance(v, (int,float))).all():
        df[c] = pd.to_numeric(df[c], errors='coerce')

for c in df.columns:
    if c.startswith("attempts_array_") or c.startswith("attempts_"):
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).round().astype(int)

for c in df.columns:
    if c.startswith("attendance_array_") or c.startswith("attendance_"):
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(float).clip(0, 100)

for c in df.columns:
    if "fee" in c.lower():
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(float).clip(lower=0)

# Fix target
if TARGET in df.columns:
    df[TARGET] = pd.to_numeric(df[TARGET], errors='coerce')
    # if it's boolean-ish or float, convert to int 0/1
    df[TARGET] = df[TARGET].fillna(0).round().astype(int)
else:
    raise KeyError(f"Target column '{TARGET}' not found in dataframe")

obj_cols = df.select_dtypes(include=['object']).columns.tolist()
if obj_cols:
    print("Warning: dropping object cols (not numeric/categorical):", obj_cols)
    df = df.drop(columns=obj_cols)

print("After preprocessing shape:", df.shape)
print("Dtypes preview:\n", df.dtypes.head(30))

y = df[TARGET]
X = df.drop(columns=[TARGET])

# Optionally drop or encode other unwanted columns (like student_id if present)
for c in ["student_id", "index"]:
    if c in X.columns:
        X = X.drop(columns=[c])

After preprocessing shape: (5000, 64)
Dtypes preview:
 Unnamed: 0                int64
dropout                   int64
slope_marks             float64
pct_change_marks        float64
momentum_marks          float64
deviation_marks         float64
slope_attendance        float64
deviation_attendance    float64
marks_mean_1            float64
marks_mean_2            float64
marks_mean_3            float64
marks_mean_4            float64
marks_mean_5            float64
marks_mean_6            float64
marks_mean_7            float64
marks_mean_8            float64
marks_std_1             float64
marks_std_2             float64
marks_std_3             float64
marks_std_4             float64
marks_std_5             float64
marks_std_6             float64
marks_std_7             float64
marks_std_8             float64
marks_min_1             float64
marks_min_2             float64
marks_min_3             float64
marks_min_4             float64
marks_min_5             float64
marks_min_6      

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train, y_train)
preds = model.predict(X_test)




/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:42:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


##Model Evaluation


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[711  39]
 [ 94 156]]


In [ ]:
from sklearn.metrics import precision_score, recall_score,f1_score

precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
accuracy = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

Accuracy: 0.867
Precision: 0.8
Recall: 0.624
F1: 0.701123595505618


##Testing

In [ ]:
# in Colab, after training:
import joblib
# if using sklearn API wrapper (XGBClassifier / XGBRegressor)
joblib.dump(model, "xgb_model.pkl")
# OR if you used xgboost.Booster:
model.save_model("xgb_model.json")   # or .bin
from google.colab import files
files.download("xgb_model.pkl")      # or xgb_model.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install bentoml


INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninsta

In [ ]:
# locally: create service file and containerize via bentoml
# then: bentoml containerize my_xgb:latest  # builds docker for you
# bentoml serve my_xgb:latest --production
import bentoml
# model is XGBClassifier() or XGBRegressor() fitted via sklearn API
bento_tag = bentoml.xgboost.save_model("my_xgb_sklearn", model)
print("Saved:", bento_tag)



Saved: Model(tag="my_xgb_sklearn:6drwrlwu4oyoyasc")


In [26]:
input_df = pd.read_excel("input.xlsx")

In [27]:
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)


In [28]:
input_label = input_df[["marks_mean", "marks_std", "marks_min", "marks_max", "attendance_array", "attempts_array" ,"fee_late_due_date_array" , "slope_marks", "pct_change_marks", "momentum_marks", "deviation_marks", "slope_attendance", "deviation_attendance"]]

In [29]:
risk = model.predict_proba(X_test)[:, 1]


In [30]:
print(risk)

[0.0149719  0.7347532  0.25930032 0.14177817 0.8139407  0.12322515
 0.28287828 0.6579853  0.06633209 0.00179997 0.03024475 0.4195049
 0.01367084 0.37962687 0.75660634 0.9132067  0.07655764 0.9552436
 0.00427316 0.08115702 0.04740329 0.00449684 0.19750458 0.00967783
 0.5660904  0.8644779  0.09921649 0.06979565 0.00630226 0.6293582
 0.08624846 0.2168441  0.04306683 0.27014318 0.04513966 0.01505163
 0.22902541 0.8207244  0.06581812 0.03377028 0.7688099  0.01469891
 0.34383303 0.03556952 0.0037289  0.02851599 0.2965044  0.00881427
 0.00650082 0.03357323 0.35981935 0.0790484  0.11759458 0.01910366
 0.03354961 0.00393891 0.25303492 0.00710561 0.96223456 0.05151544
 0.7200824  0.1923016  0.17780928 0.0151923  0.36432076 0.01048128
 0.16489917 0.3721694  0.00309435 0.04466356 0.03816414 0.78999174
 0.00680279 0.06519594 0.07423826 0.22198991 0.01711462 0.7387135
 0.01548829 0.63996536 0.10305829 0.01648845 0.76322335 0.62902176
 0.02489321 0.02422086 0.4440036  0.06080087 0.03104162 0.03146337